## Discovery and Ordering Workflow

In this example workflow, a user will be shown how to:
* Search STAC items
* View pipelines
* Validate an order
* Check an order's estimated usage
* Place an order
* Cancel an order

#### Authentication
For a more detailed view of how to authorize with MGP, please see the *Auth and Token Service* notebook in this collection

In [ ]:
import requests
import json

username = "your.email@address.com"
password= "yourpassword"

url = "https://account.maxar.com/auth/realms/mds/protocol/openid-connect/token"

payload = 'client_id=mgp&username={}&password={}&grant_type=password'.format(username, password)
headers = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.request("POST", url, headers=headers, data=payload)

access_token = {'Content-Type': 'application/json', 'Authorization': 'Bearer {}'.format(response.json()['access_token'])}

#### Discover STAC items

The first thing we want to do is search the available STAC items via Discovery. Here we will set up some variables to make the discovery process more manageable

In [ ]:
bbox = "-105,40,-104,41"
datetime = "2015-01-01T00:00:00Z/2016-01-01T00:00:00Z"
collections = "wv02"
where = "eo:cloud_cover<20"
limit = 10

We will iterate through a list of all of the ids that match our criteria and set a variable for the first id in that list

In [ ]:
stac_search_url = "https://api.maxar.com/discovery/v1/search?bbox={}&datetime={}&collections={}&where={}&limit={}&" \
                  "orderby=id".format(bbox, datetime, collections, where, limit)

stac_search_response = requests.request("GET", stac_search_url, headers=access_token)

features_list = []
for feature in stac_search_response.json()['features'][:limit]:
    features_list.append(feature['id'])
wv02_feature = features_list[0]
print(wv02_feature)

Now that we have a single feature, we can narrow down the metadata for just that feature. We want to know what the supported pipelines are for this feature, so we will narrow the result down to show the pipelines and their names

In [ ]:
stac_item_url = "https://api.maxar.com/discovery/v1/collections/{}/items/{}".format(collections, wv02_feature)

stac_item_response = requests.request("GET", stac_item_url, headers=access_token)

for i in stac_item_response.json()['links']:
    print(i['rel'])
    print(i['href'])

For this workflow, we want to use the *map-ready* pipeline. We now know that the feature can be ordered via the map-ready pipeline

If you are unfamiliar with a pipeline and what is needed for the order to work, you can use the Get a Pipeline by name request to determine what information needs to be passed in. The namespace of the pipeline can be found in the *href* link from the above function. It is the section of the link between *pipelines* and the pipeline name, in this case, with a namespace of *imagery* and a name of *map-ready*. With our pipeline identified, we can narrow down the response to show us the necessary information needed to place an order

In [ ]:
pipeline_info_url = "https://api.maxar.com/ordering/v1/pipelines/imagery/map-ready"

pipeline_info_response = requests.request("GET", pipeline_info_url, headers=access_token)

required_info = pipeline_info_response.json()['data']['settings_schema']['required']

print("The required information needed to place an order on this pipeline is: {}".format(required_info))
for info in required_info:
    type_info = pipeline_info_response.json()['data']['settings_schema']['properties'][info]
    print("The type of data for {} is: {}".format(info, type_info))

From this response we can see that the inventory_ids and the customer_description are required for order placement, and that inventory_ids is an array of strings while customer_description is a string

#### Ordering a feature

Before we fully place an order, we want to verify that the order is formatted properly and will be accepted, and what the estimated cost of the order will be. First we will set up some variables to make the requests more manageable. The output_config variable will be a dictionary where we store our S3 information, the notificaitons variable will be a list of a dictionary that will store our email information, the settings variable will be a dictionary that contains our STAC items and coordinates if we don't want to order the full item, and the metadata variable will be a dictionary that stores our project_id, or order name

In [ ]:
namespace = 'imagery'
name = 'map-ready'
output_config = {"amazon_s3": {"bucket": "yourS3BucketName", "prefix": "your/S3/Bucket/Prefix/Name"}}
settings = {
                "inventory_ids": [wv02_feature],
                "customer_description": "your order description",
                "aoi": {
                    "coordinates": [
                        [
                            [-103.991089, 39.546412],
                            [-103.900452, 39.546412],
                            [-103.900452, 39.474365],
                            [-103.991089, 39.474365],
                            [-103.991089, 39.546412]
                        ]
                    ],
                "type": "Polygon"
                }
            }
notifications = [{"type": "email", "target": "your.email@address.com", "level": "FINAL_ONLY"}]
metadata = {"project_id": "your order name"}

We will now validate the order before placing it

In [ ]:
order_validate_url = "https://api.maxar.com/ordering/v1/pipelines/{}/{}/validate".format(namespace, name)

order_validate_payload = json.dumps({"settings": settings, "output_config": output_config, "notifications": notifications, 
                                     "metadata": metadata})

order_validate_response = requests.request("POST", order_validate_url, headers=access_token, data=order_validate_payload)

print(order_validate_response.json())

The message shows *validation successful*, indicating that the order passes the validation, meaning we can place the order. One last thing to check before we place the order is to check the estimated usage of the order

In [ ]:
order_estimate_url = "https://api.maxar.com/ordering/v1/pipelines/{}/{}/estimate".format(namespace, name)

order_estimate_payload = json.dumps({"settings": settings, "output_config": output_config, "notifications": notifications, 
                                     "metadata": metadata})

order_estimate_response = requests.request("POST", order_estimate_url, headers=access_token, data=order_estimate_payload)

print(order_estimate_response.json())

From this response we can see that the order will use 62.351 sqkm of usage. We can calculate what this would cost in credits with the following workflow. We will need to obtain the ID for the product being used in the order, in this case, for the product *PRODUCT_ARCHIVE_IMAGERY_RECENT_DOWNLOAD*. We will also set our sqkm usage amount to a variable

In [ ]:
for product_id in order_estimate_response.json()['data']['usage_estimate']:
    desired_product_id = product_id['product_id']
    order_quantity = product_id['quantity']
    print("The ID for the product being utilized in the order is {}".format(desired_product_id))

With our product ID set, we now need to find the cost of this product with the associated rate table for our user. We will be using the *Tier 1 v1.0  (Default)* rate table

In [ ]:
rate_table_url = "https://api.maxar.com/account-service/api/v1/ratetables"

rate_table_response = requests.request("GET", rate_table_url, headers=access_token)

desired_rate_table = "Tier 1 v1.0  (Default)"

for rate in rate_table_response.json()['rateTables']:
    if rate['name'] == desired_rate_table:
        desired_table_id = rate['id']
print("The ID for the rate table {} is {}".format(desired_rate_table, desired_table_id))

We now have the ID associated with our rate table. We will now calculate the cost of the order by cross referencing our sqkm qunatity from the order estimate with the actual credit cost for the product being used in our order

In [ ]:
rate_product_url = "https://api.maxar.com/account-service/api/v1/ratetables/{}/productCredits".format(desired_table_id)

rate_product_response = requests.request("GET", rate_product_url, headers=access_token)

for rate_product in rate_product_response.json():
    if rate_product['product']['id'] == desired_product_id:
        print("Product {} has a cost of {} credits per {}".format(
            desired_product_id, rate_product['credits'], rate_product['creditUnitType']['description']))
        print("The order placed would consume {} credits".format(order_quantity * rate_product['credits']))

With our order cost calculated, we can now place our order using the same variables from the order validate and order estimate requests

In [ ]:
order_url = "https://api.maxar.com/ordering/v1/pipelines/{}/{}/order".format(namespace, name)

order_payload = json.dumps({"settings": settings, "output_config": output_config, "notifications": notifications, 
                            "metadata": metadata})

order_response = requests.request("POST", order_url, headers=access_token, data=order_payload)

print(order_response.json())
order_id = order_response.json()['data']['id']

After the order has been placed, we may want to check the full details of the submitted order. This can be done with the following request and passing in the order id generated from the placement of the order

In [ ]:
order_details_url = "https://api.maxar.com/ordering/v1/orders/{}".format(order_id)

order_details_response = requests.request("GET", order_details_url, headers=access_token)

print(order_details_response.json())

We also may want to occasionally check up on the status of the order. We can do this by utilizing the following request and checking the message of the response

In [ ]:
order_events_url = "https://api.maxar.com/ordering/v1/orders/{}/events".format(order_id)

order_events_response = requests.request("GET", order_events_url, headers=access_token)

print(order_events_response.json())

In some instances, we may want to cancel an order. Some pipelines unfortunately do not allow cancelling of orders. To check, we can run the get_all_pipelines function and see if cancelling is allowed for our pipeline

In [ ]:
all_pipelines_url = "https://api.maxar.com/ordering/v1/pipelines?limit=100"

all_pipelines_response = requests.request("GET", all_pipelines_url, headers=access_token)

for pipeline in all_pipelines_response.json()['data']['pipelines']:
    if pipeline['name'] == name:
        if pipeline['orders_cancellable']:
            print("Pipeline {} allows cancellations".format(name))
        else:
            print("Pipeline {} does not allow cancellations".format(name))

Assuming the pipeline allows for cancellations, we can run the cancel_order function to cancel the desired order

In [ ]:
cancel_order_url = "https://api.maxar.com/ordering/v1/order/{}/cancel".format(order_id)

cancel_order_response = requests.request("POST", cancel_order_url, headers=access_token)

print(cancel_order_response.json())